In [1]:
import csv
import os
import glob
import multiprocessing as mp
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
from scipy.interpolate import interp1d
pd.set_option('display.float_format', lambda x: '%.3f' % x)
def tf(x):
    return(np.log10(x))
import pickle as pkl

def temp_reduce(x):
    return(x-1)

def decader(x):
    return(x -x%10)
from sklearn.preprocessing import Normalizer
fourgrampath="4grams/"
fourgram_files=glob.glob(os.path.join(fourgrampath, "*.txt"))
from itertools import groupby
from operator import itemgetter
import pickle as pkl
import re
def patternmaker(x):
    x=np.array(x.notnull())
    x=x.astype(int)
    #print(x)
    val = ''.join(map(str, x))
    #print(val)
    return val

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from sklearn.decomposition import PCA,TruncatedSVD,NMF

modifier_list = pkl.load( open( "modifier_list.p", "rb" ) )
head_list = pkl.load( open( "head_list.p", "rb" ) )
words=list(set(modifier_list).union(head_list))
print(len(words))

def lemma_maker(x, y):
    #print(x,y)
    return lemmatizer.lemmatize(x,y)

br_to_us=pd.read_excel("Book.xlsx")
br_to_us_dict=dict(zip(br_to_us.UK.tolist(),br_to_us.US.tolist()))

pos_dict=dict(zip(['i', 'c','p', 't', 'm', 'r', 'd', 'j', 'f', 'x', 'e',
       'u', 'b'],['n', 'n', 'n', 'n', 'n', 'n', 'n', 'a', 'n', 'n', 'n','n', 'n']))
replacements={'r1_PoS':pos_dict,'r2_PoS':pos_dict,'l1_PoS':pos_dict,'l2_PoS':pos_dict,'r1':br_to_us_dict,'r2':br_to_us_dict,'l1':br_to_us_dict,'l2':br_to_us_dict,'modifier':br_to_us_dict,'head':br_to_us_dict}

18979


In [2]:
leftgram   = pd.concat((pd.read_csv(f,header=None,encoding="cp1252",delim_whitespace=True) for f in fourgram_files),ignore_index =True)
leftgram.columns=['freq','modifier','head','r1','r2','mod_pos','head_pos','r1_PoS','r2_PoS','decade']
modifier_noun_tags=["nn","nn1","nn2"]
head_noun_tags=["nn","nn1","nn2"]

In [3]:
leftgram=leftgram[leftgram.mod_pos.isin(modifier_noun_tags) & leftgram.head_pos.isin(head_noun_tags)]
#leftgram=leftgram[~leftgram['decade'].isin([19,20])]
leftgram=leftgram[~leftgram['r1_PoS'].isin(head_noun_tags)]
leftgram['modifier']=leftgram['modifier'].str.lower()
leftgram['head']=leftgram['head'].str.lower()
leftgram['r1']=leftgram['r1'].str.lower()
leftgram['r2']=leftgram['r2'].str.lower()
leftgram['r1_PoS']=leftgram.r1_PoS.str[0]
leftgram['r2_PoS']=leftgram.r2_PoS.str[0]
leftgram['mod_pos']=leftgram.mod_pos.str[0]
leftgram['head_pos']=leftgram.head_pos.str[0]
leftgram.replace(replacements,inplace=True)
leftgram['modifier']=np.vectorize(lemma_maker)(leftgram['modifier'], leftgram['mod_pos'])
leftgram['head']=np.vectorize(lemma_maker)(leftgram['head'], leftgram['head_pos'])
leftgram.dropna(inplace=True)
leftgram['r1']=np.vectorize(lemma_maker)(leftgram['r1'], leftgram['r1_PoS'])
leftgram['r2']=np.vectorize(lemma_maker)(leftgram['r2'], leftgram['r2_PoS'])
leftgram['modifier']=leftgram['modifier']+"_n"
leftgram['head']=leftgram['head']+"_n"
leftgram['r1']=leftgram['r1']+"_"+leftgram['r1_PoS']
leftgram['r2']=leftgram['r2']+"_"+leftgram['r2_PoS']
leftgram.drop(['mod_pos','head_pos',"r1_PoS","r2_PoS"],axis=1,inplace=True)
leftgram=leftgram.groupby(['modifier','head','r1','r2','decade'])['freq'].sum().to_frame()
display(leftgram.shape)
leftgram.head(10)

(69504, 1)

freq
modifier    head       r1     r2       decade      
abbey_n     theater_n  in_n   dublin_n 11         2
                                       13         1
                                       14         1
                                       18         2
abolition_n doctrine_n tend_v rather_n 8          1
                                       11         1
                                       12         1
                                       16         1
ac_n        cording_n  to_n   the_a    3          1
                                       4          1

In [4]:
compound_left_counts=pd.melt(leftgram.reset_index(),id_vars=['modifier','head','decade','freq'],value_vars=['r1','r2'])
compound_left=pd.pivot_table(compound_left_counts,index=['modifier','head','decade'],columns='value',values='freq',aggfunc=np.sum)
display(compound_left.shape)
compound_left.head(10)

(41624, 3715)

value                          a-mold'ring_v  a-mouldering_v  a_a  a_n  \
modifier    head       decade                                            
abbey_n     theater_n  11                nan             nan  nan  nan   
                       13                nan             nan  nan  nan   
                       14                nan             nan  nan  nan   
                       18                nan             nan  nan  nan   
abolition_n doctrine_n 8                 nan             nan  nan  nan   
                       11                nan             nan  nan  nan   
                       12                nan             nan  nan  nan   
                       16                nan             nan  nan  nan   
ac_n        cording_n  3                 nan             nan  nan  nan   
                       4                 nan             nan  nan  nan   

value                          aage_n  abbott_n  abe_n  abide_v  abilene_n  \
modifier    head       decade                                                
abbey_n     theater_n  11         nan       nan    nan      nan        nan   
                       13         nan       nan    nan      nan        nan   
                       14         nan       nan    nan      nan        nan   
                       18         nan       nan    nan      nan        nan   
abolition_n doctrine_n 8          nan       nan    nan      nan        nan   
                       11         nan       nan    nan      nan        nan   
                       12         nan       nan    nan      nan        nan   
                       16         nan       nan    nan      nan        nan   
ac_n        cording_n  3          nan       nan    nan      nan        nan   
                       4          nan       nan    nan      nan        nan   

value                          ability_n  ...    youth_n  yucatan_n  \
modifier    head       decade             ...                         
abbey_n     theater_n  11            nan  ...        nan        nan   
                       13            nan  ...        nan        nan   
                       14            nan  ...        nan        nan   
                       18            nan  ...        nan        nan   
abolition_n doctrine_n 8             nan  ...        nan        nan   
                       11            nan  ...        nan        nan   
                       12            nan  ...        nan        nan   
                       16            nan  ...        nan        nan   
ac_n        cording_n  3             nan  ...        nan        nan   
                       4             nan  ...        nan        nan   

value                          yugoslavia_n  zaki_n  zaretzki_n  zbigniew_n  \
modifier    head       decade                                                 
abbey_n     theater_n  11               nan     nan         nan         nan   
                       13               nan     nan         nan         nan   
                       14               nan     nan         nan         nan   
                       18               nan     nan         nan         nan   
abolition_n doctrine_n 8                nan     nan         nan         nan   
                       11               nan     nan         nan         nan   
                       12               nan     nan         nan         nan   
                       16               nan     nan         nan         nan   
ac_n        cording_n  3                nan     nan         nan         nan   
                       4                nan     nan         nan         nan   

value                          zest_n  ziegler_n  zoo_n  zor_n  
modifier    head       decade                                   
abbey_n     theater_n  11         nan        nan    nan    nan  
                       13         nan        nan    nan    nan  
                       14         nan        nan    nan    nan  
                       18         nan        na

In [5]:
rightgram   = pd.concat((pd.read_csv(f,header=None,encoding="cp1252",delim_whitespace=True) for f in fourgram_files),ignore_index =True)
rightgram.columns=['freq','l1','l2','modifier','head','l1_PoS','l2_PoS','mod_pos','head_pos','decade']
rightgram=rightgram[rightgram.mod_pos.isin(modifier_noun_tags) & rightgram.head_pos.isin(head_noun_tags)]
#rightgram=rightgram[rightgram['decade'].isin([19,20])]
rightgram=rightgram[~rightgram['l2_PoS'].isin(head_noun_tags)]
rightgram['modifier']=rightgram['modifier'].str.lower()
rightgram['head']=rightgram['head'].str.lower()
rightgram['l1']=rightgram['l1'].str.lower()
rightgram['l2']=rightgram['l2'].str.lower()
rightgram['l1_PoS']=rightgram.l1_PoS.str[0]
rightgram['l2_PoS']=rightgram.l2_PoS.str[0]
rightgram['mod_pos']=rightgram.mod_pos.str[0]
rightgram['head_pos']=rightgram.head_pos.str[0]
rightgram.replace(replacements,inplace=True)
rightgram['modifier']=np.vectorize(lemma_maker)(rightgram['modifier'], rightgram['mod_pos'])
rightgram['head']=np.vectorize(lemma_maker)(rightgram['head'], rightgram['head_pos'])
rightgram.dropna(inplace=True)
rightgram['l1']=np.vectorize(lemma_maker)(rightgram['l1'], rightgram['l1_PoS'])
rightgram['l2']=np.vectorize(lemma_maker)(rightgram['l2'], rightgram['l2_PoS'])
rightgram['modifier']=rightgram['modifier']+"_n"
rightgram['head']=rightgram['head']+"_n"
rightgram['l1']=rightgram['l1']+"_"+rightgram['l1_PoS']
rightgram['l2']=rightgram['l2']+"_"+rightgram['l2_PoS']
rightgram.drop(['mod_pos','head_pos',"l1_PoS","l2_PoS"],axis=1,inplace=True)
rightgram=rightgram.groupby(['modifier','head','l1','l2','decade'])['freq'].sum().to_frame()
display(rightgram.shape)
rightgram.head(10)

(242910, 1)

freq
modifier  head         l1     l2    decade      
aap_n     world_n      of_n   the_a 20         3
ab_n      iniquitate_n lava_n me_n  15         1
                                    16         2
aba_n     journal_n    for_n  the_a 19         1
                                    20         5
                       of_n   the_a 20         3
          section_n    of_n   the_a 19         1
                                    20         2
abalone_n shell_n      of_n   an_a  12         2
                                    19         1

In [6]:
compound_right_counts=pd.melt(rightgram.reset_index(),id_vars=['modifier','head','decade','freq'],value_vars=['l1','l2'])
compound_right=pd.pivot_table(compound_right_counts,index=['modifier','head','decade'],columns='value',values='freq',aggfunc=np.sum)
display(compound_right.shape)
compound_right.head(10)

(117137, 6391)

value                          a-way_a  a_a  a_n  aba_n  abandon_v  \
modifier  head         decade                                        
aap_n     world_n      20          nan  nan  nan    nan        nan   
ab_n      iniquitate_n 15          nan  nan  nan    nan        nan   
                       16          nan  nan  nan    nan        nan   
aba_n     journal_n    19          nan  nan  nan    nan        nan   
                       20          nan  nan  nan    nan        nan   
          section_n    19          nan  nan  nan    nan        nan   
                       20          nan  nan  nan    nan        nan   
abalone_n shell_n      12          nan  nan  nan    nan        nan   
                       19          nan  nan  nan    nan        nan   
abbey_n   church_n     10          nan  nan  nan    nan        nan   

value                          abandoned_a  abbott_n  abc_n  abductor_n  \
modifier  head         decade                                             
aap_n     world_n      20              nan       nan    nan         nan   
ab_n      iniquitate_n 15              nan       nan    nan         nan   
                       16              nan       nan    nan         nan   
aba_n     journal_n    19              nan       nan    nan         nan   
                       20              nan       nan    nan         nan   
          section_n    19              nan       nan    nan         nan   
                       20              nan       nan    nan         nan   
abalone_n shell_n      12              nan       nan    nan         nan   
                       19              nan       nan    nan         nan   
abbey_n   church_n     10              nan       nan    nan         nan   

value                          abelard_n   ...    youth_n  yukon_n  zev_n  \
modifier  head         decade              ...                              
aap_n     world_n      20            nan   ...        nan      nan    nan   
ab_n      iniquitate_n 15            nan   ...        nan      nan    nan   
                       16            nan   ...        nan      nan    nan   
aba_n     journal_n    19            nan   ...        nan      nan    nan   
                       20            nan   ...        nan      nan    nan   
          section_n    19            nan   ...        nan      nan    nan   
                       20            nan   ...        nan      nan    nan   
abalone_n shell_n      12            nan   ...        nan      nan    nan   
                       19            nan   ...        nan      nan    nan   
abbey_n   church_n     10            nan   ...        nan      nan    nan   

value                          zinc_n  zion_n  zip-top_a  zip_v  \
modifier  head         decade                                     
aap_n     world_n      20         nan     nan        nan    nan   
ab_n      iniquitate_n 15         nan     nan        nan    nan   
                       16         nan     nan        nan    nan   
aba_n     journal_n    19         nan     nan        nan    nan   
                       20         nan     nan        nan    nan   
          section_n    19         nan     nan        nan    nan   
                       20         nan     nan        nan    nan   
abalone_n shell_n      12         nan     nan        nan    nan   
                       19         nan     nan        nan    nan   
abbey_n   church_n     10         nan     nan        nan    nan   

value                          zipper-lock_a  zoltan_a  zone_n  
modifier  head         decade                                   
aap_n     world_n      20                nan       nan     nan  
ab_n      iniquitate_n 15                nan       nan     nan  
                       16                nan       nan     nan  
aba_n     journal_n    19                nan       nan     nan  
                       20                nan       nan     nan  
          section_n    19                nan       nan     nan  
        

In [7]:
compounds=pd.concat([compound_left_counts,compound_right_counts])
compounds=pd.pivot_table(compounds,index=['modifier','head','decade'],columns='value',values='freq',aggfunc=np.sum)
display(compounds.shape)
compounds.head(10)

(132715, 8168)

value                          a-mold'ring_v  a-mouldering_v  a-way_a  a_a  \
modifier  head         decade                                                
aap_n     world_n      20                nan             nan      nan  nan   
ab_n      iniquitate_n 15                nan             nan      nan  nan   
                       16                nan             nan      nan  nan   
aba_n     journal_n    19                nan             nan      nan  nan   
                       20                nan             nan      nan  nan   
          section_n    19                nan             nan      nan  nan   
                       20                nan             nan      nan  nan   
abalone_n shell_n      12                nan             nan      nan  nan   
                       19                nan             nan      nan  nan   
abbey_n   church_n     10                nan             nan      nan  nan   

value                          a_n  aage_n  aba_n  abandon_v  abandoned_a  \
modifier  head         decade                                               
aap_n     world_n      20      nan     nan    nan        nan          nan   
ab_n      iniquitate_n 15      nan     nan    nan        nan          nan   
                       16      nan     nan    nan        nan          nan   
aba_n     journal_n    19      nan     nan    nan        nan          nan   
                       20      nan     nan    nan        nan          nan   
          section_n    19      nan     nan    nan        nan          nan   
                       20      nan     nan    nan        nan          nan   
abalone_n shell_n      12      nan     nan    nan        nan          nan   
                       19      nan     nan    nan        nan          nan   
abbey_n   church_n     10      nan     nan    nan        nan          nan   

value                          abbott_n  ...    ziegler_n  zinc_n  zion_n  \
modifier  head         decade            ...                                
aap_n     world_n      20           nan  ...          nan     nan     nan   
ab_n      iniquitate_n 15           nan  ...          nan     nan     nan   
                       16           nan  ...          nan     nan     nan   
aba_n     journal_n    19           nan  ...          nan     nan     nan   
                       20           nan  ...          nan     nan     nan   
          section_n    19           nan  ...          nan     nan     nan   
                       20           nan  ...          nan     nan     nan   
abalone_n shell_n      12           nan  ...          nan     nan     nan   
                       19           nan  ...          nan     nan     nan   
abbey_n   church_n     10           nan  ...          nan     nan     nan   

value                          zip-top_a  zip_v  zipper-lock_a  zoltan_a  \
modifier  head         decade                                              
aap_n     world_n      20            nan    nan            nan       nan   
ab_n      iniquitate_n 15            nan    nan            nan       nan   
                       16            nan    nan            nan       nan   
aba_n     journal_n    19            nan    nan            nan       nan   
                       20            nan    nan            nan       nan   
          section_n    19            nan    nan            nan       nan   
                       20            nan    nan            nan       nan   
abalone_n shell_n      12            nan    nan            nan       nan   
                       19            nan    nan            nan       nan   
abbey_n   church_n     10            nan    nan            nan       nan   

value                          zone_n  zoo_n  zor_n  
modifier  head         decade                        
aap_n     world_n      20         nan    nan    nan  
ab_n      iniquitate_n 15         nan    nan    nan  
                       16         nan    nan    nan  
aba_n     journal_n    19     

In [8]:
compounds_list=list(set(compounds.reset_index(level="decade").index.tolist()))

In [9]:
len(compounds_list)

32488

In [10]:
pkl.dump( compounds_list, open( "compounds.p", "wb" ) )